# Generate Layout 

The first step is to create a layout and test manifest for your project, so that you can relate measurements and analyses back to your source layout.

You can generate the layout with any layout tool. In this notebook you can use kfactory.

## Generate layout

Kfactory allows you to write metadata directly into the cell

You can download this [generate_layout.py](./generate_layout.py) file

In [ ]:
import csv

from generate_layout import top

c = top()
c.write_gds("test_chip.gds")
c.plot()

## Generate device table

For generating the device table you need to use Kfactory.

A device needs the X, Y position on the layout.
Note that you can instantiate a Cell many times into a die.

In [ ]:
sweep = c.kcl["cutback"]
csvpath = "device_table.csv"

analysis = "[cutback]"
analysis_parameters = "[{}]"
length = 20

with open(csvpath, "w") as f:
    writer = csv.writer(f)
    writer.writerow(
        [
            "cell",
            "x",
            "y",
            "components",
            # "analysis",
            # "analysis_parameters",
        ]
    )
    iterator = sweep.kdb_cell.begin_instances_rec()
    iterator.targets = "cutback_loss*"
    while not iterator.at_end():
        _c = c.kcl[iterator.inst_cell().cell_index()]
        name = _c.name
        _disp = (iterator.trans() * iterator.inst_trans()).disp
        params = name.split("-")  # we extract the width from the device name
        writer.writerow(
            [
                name,
                _disp.x,
                _disp.y,
                _c.info["components"],
                # analysis,
                # analysis_parameters,
            ]
        )
        iterator.next()

You can take a look a the contents of the device manifest you created.

In [ ]:
import pandas as pd

dt = pd.read_csv(csvpath)
dt

In [ ]:
dt.to_csv("device_table.csv", index=False)